In [1]:
# Import required libraries/code
import seaborn as sns
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torchmetrics
from torch import nn
from torchvision import transforms, datasets
from timeit import default_timer as timer
from torchinfo import summary
from fvcore.nn import FlopCountAnalysis, flop_count_table
from torchvision.models.segmentation import deeplabv3_resnet101

import os

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import wandb
import deeplabv2

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

if __name__ != '__main__':
    raise Exception("This script should not be imported; it should be run directly.")

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)


# Create a transforms pipeline
simple_transform = transforms.Compose([
    transforms.Resize((512, 256)),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Create training and testing DataLoader's as well as get a list of class names

data_path = "Cityscapes/Cityspaces/gtFine"
train_dset = ImageFolder(f"{data_path}/train", transform=simple_transform)
val_dset = ImageFolder(f"{data_path}/val", transform=simple_transform)
class_names = list(os.listdir(f"{data_path}/train"))  # 'pizza', 'steak', 'sushi'
# print(f"Class names: {class_names}")

train_dataloader = DataLoader(train_dset, batch_size=2, num_workers=2, shuffle=True)
val_dataloader = DataLoader(val_dset, batch_size=2, num_workers=2)

# train_dataloader, test_dataloader, class_names

In [2]:
model_0 = deeplabv2.get_deeplab_v2(num_classes=10,pretrain_model_path='deepLab_resnet_pretrained_imagenet.pth')

Deeplab pretraining loading...


In [3]:
def train(model, optimizer_train, dataloader, loss_fn_train):
    
    model.train()  # Set the model to training mode
    train_loss = 0.0
    correct = 0
    total = 0

    for inputs_train, targets_train in dataloader:
        inputs_train, targets_train = inputs_train.to(device), targets_train.to(
            device)  # Move data to the appropriate device

        optimizer_train.zero_grad()  # Zero out gradients from the previous iteration

        # problem satarts here
        outputs_train, _, _  = model(inputs_train)  # Forward pass
        # Calculate the loss
        # outputs_train = outputs_train.view(outputs_train.size(0), outputs_train.size(1), -1)
        # outputs_train = torch.mean(outputs_train, dim=2)
        
        loss = loss_fn_train(outputs_train, targets_train)  # Calculate the loss
        
        loss.backward()  # Backward pass
        optimizer_train.step()  # Update the weights

        train_loss += loss.item() * inputs_train.size(0)  # Accumulate the total loss
        _, predicted_train = outputs_train.max(1)
        total += targets_train.size(0)
        correct += predicted_train.eq(targets_train).sum().item()

    # Calculate average loss and accuracy for the epoch
    avg_loss = train_loss / len(dataloader.dataset)
    accuracy = correct / total
    
    return avg_loss, accuracy


def test(model, dataloader, loss_fn):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation during inference
        for inputs_test, targets_test in dataloader:
            inputs_test, targets_test = inputs_test.to(device), targets_test.to(device)

            outputs_test = model(inputs_test)  # Forward pass
            # Calculate the loss
            outputs_test = outputs_test.view(outputs_test.size(0), outputs_test.size(1), -1)
            outputs_test = torch.mean(outputs_test, dim=2)
            loss = loss_fn(outputs_test, targets_test)  # Calculate the loss

            test_loss += loss.item() * inputs_test.size(0)  # Accumulate the total loss
            _, predicted_test = outputs_test.max(1)
            total += targets_test.size(0)
            correct += predicted_test.eq(targets_test).sum().item()

    # Calculate average loss and accuracy for the test set
    avg_loss = test_loss / len(dataloader.dataset)
    accuracy = correct / total

    return avg_loss, accuracy



model_0.to(device)

# Define your loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr=1e-3)
        

C:\Users\Luca\PycharmProjects\MLDL\venv\Lib\site-packages\torch\nn\modules\module.py:1150: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:30.)
  return t.to(device, dtype if t.is_floating_point() or t.is_complex() else None, non_blocking)


In [4]:
start_time = timer()

# Setup training and save the results
for _ in range(50):
    train(model_0, optimizer, train_dataloader, loss_fn)
    test_acc = test(model_0, val_dataloader, loss_fn)
    print(f"Test accuracy: {test_acc}")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")

Test accuracy: (2.357779986381531, 0.0)


KeyboardInterrupt: 